In [ ]:
!pip3 install neattext
import pandas as pd
import neattext.functions as nfx
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = api_token = {"username":"<Username>","key":"<API_KEY>"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d thedevastator/udemy-courses-revenue-generation-and-course-anal
!unzip udemy-courses-revenue-generation-and-course-anal.zip

  0% 0.00/429k [00:00<?, ?B/s]
100% 429k/429k [00:00<00:00, 146MB/s]
Archive:  udemy-courses-revenue-generation-and-course-anal.zip
  inflating: 3.1-data-sheet-udemy-courses-business-courses.csv  
  inflating: 3.1-data-sheet-udemy-courses-design-courses.csv  
  inflating: 3.1-data-sheet-udemy-courses-music-courses.csv  
  inflating: 3.1-data-sheet-udemy-courses-web-development.csv  
  inflating: Entry Level Project Sheet - 3.1-data-sheet-udemy-courses-web-development.csv  


In [ ]:
business_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-business-courses.csv')
design_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-design-courses.csv')
music_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-music-courses.csv')
web_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-web-development.csv')
entry_df = pd.read_csv('/content/Entry Level Project Sheet - 3.1-data-sheet-udemy-courses-web-development.csv')
df = pd.concat([business_df, design_df,music_df,web_df,entry_df])
df.head()

,course_id,course_title,url,price,num_subscribers,num_reviews,num_lectures,level,Rating,content_duration,published_timestamp,subject,rating,Date,Free/Paid
0,49798.0,Bitcoin or How I Learned to Stop Worrying and ...,https://www.udemy.com/bitcoin-or-how-i-learned...,0.0,65576.0,936.0,24.0,All Levels,0.56,8.0,2013-04-20T02:25:22Z,Business Finance,NaN,NaN,NaN
1,48841.0,Accounting in 60 Minutes - A Brief Introduction,https://www.udemy.com/accounting-in-60-minutes...,0.0,56659.0,4397.0,16.0,Beginner Level,0.95,1.5,2013-04-07T21:39:25Z,Business Finance,NaN,NaN,NaN
2,133536.0,Stock Market Investing for Beginners,https://www.udemy.com/the-beginners-guide-to-t...,0.0,50855.0,2698.0,15.0,All Levels,0.91,1.5,2013-12-25T19:53:34Z,Business Finance,NaN,NaN,NaN
3,151668.0,Introduction to Financial Modeling,https://www.udemy.com/financial-modeling-asimp...,0.0,29167.0,1463.0,8.0,All Levels,0.18,1.5,2014-05-27T16:22:16Z,Business Finance,NaN,NaN,NaN
4,648826.0,The Complete Financial Analyst Course 2017,https://www.udemy.com/the-complete-financial-a...,195.0,24481.0,2347.0,174.0,All Levels,0.37,10.0,2016-01-21T01:38:48Z,Business Finance,NaN,NaN,NaN


In [ ]:
 df['course_title'].iloc[1:5]

1    Accounting in 60 Minutes - A Brief Introduction
2               Stock Market Investing for Beginners
3                 Introduction to Financial Modeling
4         The Complete Financial Analyst Course 2017
Name: course_title, dtype: object

**Remove Stopwords**


**Remove special characters**

In [ ]:
df = df.drop_duplicates(subset=['course_title'])

df['clean_title'] = df['course_title'].astype(str)

df['clean_title'] = df['clean_title'].apply(nfx.remove_stopwords)

df['clean_title'] = df['clean_title'].apply(nfx.remove_special_characters)

df['clean_title'].iloc[1:5]

<ipython-input-57-ec6eb02415fd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_title'] = df['course_title'].astype(str)
<ipython-input-57-ec6eb02415fd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_title'] = df['clean_title'].apply(nfx.remove_stopwords)
<ipython-input-57-ec6eb02415fd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

1    Accounting 60 Minutes  Brief Introduction
2             Stock Market Investing Beginners
3              Introduction Financial Modeling
4       Complete Financial Analyst Course 2017
Name: clean_title, dtype: object

In [ ]:
countvect = CountVectorizer()

cv_mat = countvect.fit_transform(df['clean_title'])

cv_mat

<3704x3560 sparse matrix of type '<class 'numpy.int64'>'
	with 18507 stored elements in Compressed Sparse Row format>

In [ ]:
df_cv_words = pd.DataFrame(cv_mat.todense(),columns=countvect.get_feature_names())

df_cv_words

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000005,001,01,02,10,100,101,101master,102,10k,...,zend,zero,zerotohero,zf2,zinsen,zoho,zombie,zu,zuhause,zur
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
cosine_sim_mat = cosine_similarity(cv_mat)

In [ ]:
def recommend_course(title,numrec = 10):
    
    course_index = pd.Series( df.index, index=df['clean_title']).drop_duplicates()
    
    index = course_index[title]

    scores = list(enumerate(cosine_sim_mat [index]))
    
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    selected_course_index = [i[0] for i in sorted_scores[1:]]

    selected_course_score = [i[1] for i in sorted_scores[1:]]

    rec_df = df.iloc[selected_course_index]

    rec_df['Similarity_Score'] = selected_course_score

    final_recommended_courses = rec_df[[
        'course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']]

    return final_recommended_courses.head(numrec)




ans = recommend_course('Introduction Financial Modeling',20)

ans

<ipython-input-66-c8e3e1b10513>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_df['Similarity_Score'] = selected_course_score


,course_title,Similarity_Score,url,price,num_subscribers
426,Financial Modeling 101,0.666667,https://www.udemy.com/financial-modeling-101/,0.0,1086.0
430,Intro to Financial Modeling,0.666667,https://www.udemy.com/intro-to-financial-model...,105.0,1077.0
809,Introduction to Financial Statements,0.666667,https://www.udemy.com/introduction-to-financia...,40.0,112.0
10,Financial Accounting - A Brief Introduction,0.577350,https://www.udemy.com/accounting-in-less-then-...,0.0,17847.0
322,Introduction to Financial Statement Analysis,0.577350,https://www.udemy.com/introduction-to-financia...,20.0,1480.0
346,Financial Modeling in Excel for Startups,0.577350,https://www.udemy.com/financial-modeling-in-ex...,50.0,1334.0
383,CFP - Introduction to Financial Planning,0.577350,https://www.udemy.com/apnacoursecfp/,30.0,1192.0
709,Financial Modeling for Professionals in 1 Day!,0.577350,https://www.udemy.com/financial-modeling-for-p...,200.0,226.0
1002,Introduction to Financial Consolidation under ...,0.577350,https://www.udemy.com/introduction-to-financia...,25.0,21.0
1133,Learn Financial Modeling from Scratch - Part 2,0.577350,https://www.udemy.com/learn-financial-modeling...,50.0,2.0


In [ ]:
ans.to_json(orient ='index')

In [ ]:
class RecommendCourses:
  def __init__(self):
    #create a dataframe 
    business_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-business-courses.csv')
    design_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-design-courses.csv')
    music_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-music-courses.csv')
    web_df = pd.read_csv('/content/3.1-data-sheet-udemy-courses-web-development.csv')
    entry_df = pd.read_csv('/content/Entry Level Project Sheet - 3.1-data-sheet-udemy-courses-web-development.csv')
    df = pd.concat([business_df, design_df,music_df,web_df,entry_df])

    #drop duplicates from the course_title column
    df = df.drop_duplicates(subset=['course_title'])
    # clean_title column to string from course_title
    df['clean_title'] = df['course_title'].astype(str)
    # clean_title remove stopwords
    df['clean_title'] = df['clean_title'].apply(nfx.remove_stopwords)
    # clean_title remove special characters
    df['clean_title'] = df['clean_title'].apply(nfx.remove_special_characters)

    # create CountVectorizer
    countvect = CountVectorizer()
    cv_mat = countvect.fit_transform(df['clean_title'])

    # create CV words
    df_cv_words = pd.DataFrame(cv_mat.todense(),columns=countvect.get_feature_names())
    self.df = df
    self.cosine_sim_mat = cosine_similarity(cv_mat)
    print("Data loaded Successfully")


  def recommend_course(self,title,numrec = 10):
    course_index = pd.Series( df.index, index=df['clean_title']).drop_duplicates()
    if title in course_index:
      index = course_index[title]
      scores = list(enumerate(cosine_sim_mat [index]))
      sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
      selected_course_index = [i[0] for i in sorted_scores[1:]]
      selected_course_score = [i[1] for i in sorted_scores[1:]]
      rec_df = df.iloc[selected_course_index]
      rec_df['Similarity_Score'] = selected_course_score
      final_recommended_courses = rec_df[[
          'course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']]
      return {"error":None,"data":final_recommended_courses.head(numrec).to_json(orient ='index')}
    else:
      return {"error": "course with title name "+title+" not found"}

recommendation = RecommendCourses()
recommendation.recommend_course('Introduction Financial Modeling',20)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
<ipython-input-81-000ac1759785>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_df['Similarity_Score'] = selected_course_score


{'error': None,
 'data': '{"426":{"course_title":"Financial Modeling 101","Similarity_Score":0.6666666667,"url":"https:\\/\\/www.udemy.com\\/financial-modeling-101\\/","price":0.0,"num_subscribers":1086.0},"430":{"course_title":"Intro to Financial Modeling","Similarity_Score":0.6666666667,"url":"https:\\/\\/www.udemy.com\\/intro-to-financial-modeling\\/","price":105.0,"num_subscribers":1077.0},"809":{"course_title":"Introduction to Financial Statements","Similarity_Score":0.6666666667,"url":"https:\\/\\/www.udemy.com\\/introduction-to-financial-statements\\/","price":40.0,"num_subscribers":112.0},"10":{"course_title":"Financial Accounting - A Brief Introduction","Similarity_Score":0.5773502692,"url":"https:\\/\\/www.udemy.com\\/accounting-in-less-then-one-hour\\/","price":0.0,"num_subscribers":17847.0},"322":{"course_title":"Introduction to Financial Statement Analysis","Similarity_Score":0.5773502692,"url":"https:\\/\\/www.udemy.com\\/introduction-to-financial-statement-analysis\\/","